<a href="https://colab.research.google.com/github/fralfaro/MAT281/blob/main/docs/labs/lab_04.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MAT281 - Laboratorio N°04


**Objetivo**: Aplicar técnicas intermedias y avanzadas de análisis de datos con pandas utilizando un caso real: el Índice de Libertad de Prensa. Este laboratorio incluye operaciones de limpieza, transformación, combinación de datos, y análisis exploratorio usando `merge`, `groupby`, `concat` y otras funciones fundamentales.




**Descripción del Dataset**

El presente conjunto de datos está orientado al análisis del **Índice de Libertad de Prensa**, una métrica internacional que evalúa el nivel de libertad del que gozan periodistas y medios de comunicación en distintos países. Este índice es recopilado anualmente por la organización **Reporteros sin Fronteras**.

La base de datos contempla observaciones por país y año, e incluye tanto el valor del índice como el ranking correspondiente. A menor puntaje en el índice, mayor nivel de libertad de prensa.

**Diccionario de variables**

| Variable     | Clase    | Descripción                                                                          |
| ------------ | -------- | ------------------------------------------------------------------------------------ |
| `codigo_iso` | carácter | Código ISO 3166-1 alfa-3 que representa a cada país.                                 |
| `pais`       | carácter | Nombre oficial del país.                                                             |
| `anio`       | entero   | Año en que se registró la medición del índice.                                       |
| `indice`     | numérico | Valor numérico del Índice de Libertad de Prensa (menor valor indica mayor libertad). |
| `ranking`    | entero   | Posición relativa del país en el ranking mundial de libertad de prensa.              |


**Fuente original y adaptación pedagógica**

* **Fuente original**: [Reporteros sin Fronteras](https://www.rsf-es.org/), recopilado y publicado a través del portal del [Banco Mundial](https://tcdata360.worldbank.org/indicators/h3f86901f?country=BRA&indicator=32416&viz=line_chart&years=2001,2019).
* **Adaptación educativa**: Los archivos han sido modificados intencionalmente para incorporar desafíos técnicos que permiten aplicar los contenidos abordados en clases, tales como limpieza de datos, normalización, detección de duplicados, y combinación de fuentes.


**Descripción de los archivos disponibles**

* **`libertad_prensa_codigo.csv`**: Contiene los pares `codigo_iso` y `pais`. Incluye intencionalmente un código ISO con dos nombres distintos de país para efectos de limpieza y validación de datos.

* **`libertad_prensa_01.csv`**: Contiene registros de los años **anteriores a 2010**. Incluye las variables `PAIS`, `ANIO`, `INDICE`, y `RANKING` con nombres de columna en **mayúsculas**.

* **`libertad_prensa_02.csv`**: Contiene registros de los años **desde 2010 en adelante**. Estructura similar al archivo anterior, con nombres de columna también en **mayúsculas**.





In [38]:
import numpy as np
import pandas as pd

# lectura de datos
archivos_anio = [
    'https://raw.githubusercontent.com/fralfaro/MAT281/main/docs/labs/data/libertad_prensa_01.csv',
    'https://raw.githubusercontent.com/fralfaro/MAT281/main/docs/labs/data/libertad_prensa_02.csv'
 ]
df_codigos = pd.read_csv('https://raw.githubusercontent.com/fralfaro/MAT281/main/docs/labs/data/libertad_prensa_codigo.csv')



### 1. Consolidación y limpieza de datos

A partir de los archivos disponibles, realice los siguientes pasos:

**a)** Cree un DataFrame llamado `df_anio` que consolide la información proveniente de los archivos **`libertad_prensa_01.csv`** y **`libertad_prensa_02.csv`**, correspondientes a distintas ventanas de tiempo. Recuerde que ambos archivos tienen nombres de columnas en mayúscula, por lo que debe normalizarlas a **minúscula** para asegurar consistencia.

**b)** Explore el archivo **`libertad_prensa_codigo.csv`** e identifique el código ISO que aparece asociado a dos nombres de país distintos. Elimine el registro que corresponda a un valor incorrecto o inconsistente, conservando solo el que considere válido.

**c)** Una vez preparados los archivos, cree un nuevo DataFrame llamado `df` que combine `df_anio` con `df_codigos`, utilizando la columna `codigo_iso` como clave. Asegúrese de realizar una unión que conserve únicamente los registros que tengan coincidencia en ambas fuentes.

> **Sugerencia**:
>
> * Para unir los archivos por filas (años), utilice la función `pd.concat([...])`.
> * Para combinar información por columnas (variables), utilice `pd.merge(...)` especificando `on='codigo_iso'`.



In [100]:
"a)"
lp1=pd.read_csv(archivos_anio[0])
lp2=pd.read_csv(archivos_anio[1])

lp1.columns = lp1.columns.str.lower()
lp2.columns = lp2.columns.str.lower()
df_anio = pd.concat([lp1, lp2], ignore_index=True)
"b)"
df_codigos[df_codigos.duplicated('codigo_iso', keep=False)]
df_codigos = df_codigos.drop_duplicates('codigo_iso', keep='first')
"c)"
df = pd.merge(df_anio, df_codigos, on='codigo_iso', how='inner')
df.head()

,codigo_iso,anio,indice,ranking,pais
0,ZWE,2001,48.30,76.0,Zimbabue
1,ZWE,2002,45.50,84.0,Zimbabue
2,ZWE,2003,67.50,108.0,Zimbabue
3,ZWE,2004,64.25,97.0,Zimbabue
4,ZWE,2005,50.00,77.0,Zimbabue




### 2. Exploración inicial del conjunto de datos

Una vez que hayas consolidado el DataFrame final `df`, realiza un análisis exploratorio básico respondiendo las siguientes preguntas:

#### **Estructura del DataFrame**

* ¿Cuántas **filas (observaciones)** contiene el conjunto de datos?
* ¿Cuántas **columnas** tiene el DataFrame?
* ¿Cuáles son los **nombres de las columnas**?
* ¿Qué **tipo de datos** tiene cada columna?
* ¿Hay columnas con un tipo de dato inesperado (por ejemplo, fechas como strings)?

#### **Resumen estadístico**

* Genera un resumen estadístico del conjunto de datos con `.describe()`.
  ¿Qué observas sobre los valores de `indice` y `ranking`?
* ¿Qué valores mínimo, máximo y promedio tiene la columna `indice`?
* ¿Qué países presentan los valores extremos en `indice` y `ranking`?

#### **Datos faltantes**

* ¿Cuántos valores nulos hay en cada columna?
* ¿Qué proporción de observaciones tienen valores faltantes?
* ¿Hay columnas con más del 30% de datos faltantes?

#### **Unicidad y duplicados**

* ¿Cuántos países distintos (`pais`) hay en el DataFrame?
* ¿Cuántos años distintos (`anio`) hay representados?
* ¿Existen filas duplicadas (exactamente iguales)? ¿Cuántas?

#### **Validación cruzada de columnas**

* ¿Hay inconsistencias entre el país (`pais`) y su código (`codigo_iso`)?
  (por ejemplo, un mismo código ISO asociado a más de un país)

> **Sugerencia**: Apoya tu análisis con funciones como `.info()`, `.nunique()`, `.isnull().sum()`, `.duplicated()`, `.value_counts()`, entre otras.



    

In [99]:
"Estructura del DataFrame"
print(f"Hay {df.shape[0]} Filas (observaciones)")
print(f"Hay {df.shape[1]} Columnas (variables)")
print(f"El nombre de las columnas son: {list(df.columns)}")
print("Los tipos de datos por cada columna son:")
print(df.dtypes)
print("No hay columnas con un tipo de dato inesperado")

"Resumen estadístico"
display(df.describe())
print(f"El mínimo es {df['indice'].min()}")
print(f"El máximo es {df['indice'].max()}")
print(f"El promedio es {df['indice'].mean():.2f}")
print(f"La mediana es {df['indice'].median()}")

min_indice = df[df['indice'] == df['indice'].min()]
max_indice = df[df['indice'] == df['indice'].max()]
min_ranking = df[df['ranking'] == df['ranking'].min()]
max_ranking = df[df['ranking'] == df['ranking'].max()]
print(f"El pais con menor índice es{min_indice['pais'].values[0]} ({min_indice['anio'].values[0]}) y con mayor índice es {max_indice['pais'].values[0]} ({max_indice['anio'].values[0]})")
print(f"Mientras que, el que tiene menor ranking es {min_ranking['pais'].values[0]} ({min_ranking['anio'].values[0]}) y mayor ranking es {max_ranking['pais'].values[0]} ({max_ranking['anio'].values[0]})")

"Datos faltantes"
if df.isnull().sum().sum() == 1:
  print(f"Hay {df.isnull().sum().sum()} valor nulo en el DataFrame")
elif df.isnull().sum().sum() > 1:
  print(f"Hay {df.isnull().sum().sum()} valores nulos en el DataFrame")
else:
  print(f"No hay valores nulos en el DataFrame")
print("La proporción de valores nulos por columna es")
proporcion_nulos = (df.isnull().sum() / len(df)) * 100
display(proporcion_nulos.round(2))
columnas_problematicas = proporcion_nulos[proporcion_nulos > 30]
print("Las columnas con más del 30% de datos faltantes son:")
display(columnas_problematicas)

"Unicidad y duplicados"
print(f"Hay {df['pais'].nunique()} países distintos")
print(f"Hay {df['anio'].nunique()} años distintos")
print(f"Hay {df.duplicated().sum()} filas duplicadas")

"Validación cruzada de columnas"
inconsistencias = df.groupby('codigo_iso')['pais'].nunique()
print("No se encuentran inconsistencias. Luego, la información general del documento es")
display(df.info())
print(f"Las primeras 5 filas son:")
print(df.head())
print(f"Las últimas 5 filas son:")
print(df.tail())
print(f"Distribución de observaciones por año:")
display(df['anio'].value_counts().sort_index())

Hay 17 Filas (observaciones)
Hay 5 Columnas (variables)
El nombre de las columnas son: ['codigo_iso', 'anio', 'indice', 'ranking', 'pais']
Los tipos de datos por cada columna son:
codigo_iso     object
anio            int64
indice        float64
ranking       float64
pais           object
dtype: object
No hay columnas con un tipo de dato inesperado


,anio,indice,ranking
count,17.000000,16.00000,17.000000
mean,2009.941176,47.41625,105.941176
std,5.963122,9.65499,19.568695
min,2001.000000,38.12000,76.000000
25%,2005.000000,40.18250,87.000000
50%,2009.000000,43.86500,115.000000
75%,2015.000000,51.00000,124.000000
max,2019.000000,67.50000,128.000000


El mínimo es 38.12
El máximo es 67.5
El promedio es 47.42
La mediana es 43.864999999999995
El pais con menor índice esZimbabue (2012) y con mayor índice es Zimbabue (2003)
Mientras que, el que tiene menor ranking es Zimbabue (2001) y mayor ranking es Zimbabue (2017)
Hay 1 valor nulo en el DataFrame
La proporción de valores nulos por columna es


,0
codigo_iso,0.00
anio,0.00
indice,5.88
ranking,0.00
pais,0.00


Las columnas con más del 30% de datos faltantes son:


,0


Hay 1 países distintos
Hay 17 años distintos
Hay 0 filas duplicadas
No se encuentran inconsistencias. Luego, la información general del documento es
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17 entries, 0 to 16
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   codigo_iso  17 non-null     object 
 1   anio        17 non-null     int64  
 2   indice      16 non-null     float64
 3   ranking     17 non-null     float64
 4   pais        17 non-null     object 
dtypes: float64(2), int64(1), object(2)
memory usage: 812.0+ bytes


None

Las primeras 5 filas son:
  codigo_iso  anio  indice  ranking      pais
0        ZWE  2001   48.30     76.0  Zimbabue
1        ZWE  2002   45.50     84.0  Zimbabue
2        ZWE  2003   67.50    108.0  Zimbabue
3        ZWE  2004   64.25     97.0  Zimbabue
4        ZWE  2005   50.00     77.0  Zimbabue
Las últimas 5 filas son:
   codigo_iso  anio  indice  ranking      pais
12        ZWE  2015   40.41    118.0  Zimbabue
13        ZWE  2016     NaN    122.0  Zimbabue
14        ZWE  2017   41.44    128.0  Zimbabue
15        ZWE  2018   40.53    126.0  Zimbabue
16        ZWE  2019   42.23    127.0  Zimbabue
Distribución de observaciones por año:


,count
anio,
2001,1
2002,1
2003,1
2004,1
2005,1
2006,1
2007,1
2008,1
2009,1





### 3. Comparación regional: países latinoamericanos

En esta sección se busca identificar cuáles son los países de América Latina que han presentado los valores extremos del **Índice de Libertad de Prensa** en cada año observado.

> Recuerda que un menor puntaje en `indice` implica mayor libertad de prensa.

#### **Tareas:**

**a)** Utilizando un ciclo `for`, recorre cada año del conjunto de datos filtrado por países latinoamericanos, y determina para cada año:

* El país con el menor valor de `indice` (mayor libertad de prensa).
* El país con el mayor valor de `indice` (menor libertad de prensa).

**b)** Resuelve la misma tarea del punto anterior utilizando un enfoque vectorizado con `groupby`, sin usar ciclos explícitos.



#### **Lista de países latinoamericanos considerada:**

```python
america = ['ARG', 'ATG', 'BLZ', 'BOL', 'BRA', 'CAN', 'CHL', 'COL', 'CRI',
           'CUB', 'DOM', 'ECU', 'GRD', 'GTM', 'GUY', 'HND', 'HTI', 'JAM',
           'MEX', 'NIC', 'PAN', 'PER', 'PRY', 'SLV', 'SUR', 'TTO', 'URY',
           'USA', 'VEN']
```

> Puedes usar esta lista para filtrar el DataFrame final por la columna `codigo_iso`.



In [98]:
# respuesta
america = ['ARG', 'ATG', 'BLZ', 'BOL', 'BRA', 'CAN', 'CHL', 'COL', 'CRI',
       'CUB', 'DOM', 'ECU', 'GRD', 'GTM', 'GUY', 'HND', 'HTI', 'JAM',
       'MEX', 'NIC', 'PAN', 'PER', 'PRY', 'SLV', 'SUR', 'TTO', 'URY',
       'USA', 'VEN']



### 4. Análisis anual del índice por país

En esta sección se busca analizar la evolución del **índice máximo** de libertad de prensa alcanzado por cada país a lo largo del tiempo.

#### **Tarea principal:**

* Construye una tabla dinámica (`pivot_table`) donde las **filas** correspondan a los países, las **columnas** a los años (`anio`) y los **valores** sean el `indice` máximo alcanzado por cada país en ese año.
* Asegúrate de reemplazar los valores nulos resultantes con `0`.

> **Hint**: Puedes utilizar el parámetro `fill_value=0` en `pd.pivot_table(...)`.



#### **Preguntas adicionales:**

**a)** ¿Qué país tiene el mayor valor de `indice` en toda la tabla resultante? ¿Y cuál tiene el menor (distinto de cero)?
**b)** ¿Qué años presentan en promedio los valores de `indice` más altos? ¿Y los más bajos?

> (Pista: usa `.mean(axis=0)` sobre la tabla pivot)

**c)** ¿Qué país muestra mayor **variabilidad** (diferencia entre su máximo y mínimo `indice` a lo largo del tiempo)?

> (Pista: aplica `.max(axis=1) - .min(axis=1)`)

**d)** ¿Existen países con índice constante a lo largo de todos los años registrados? ¿Cuáles?

**e)** ¿Qué países no tienen ningún dato (es decir, quedaron con todos los valores igual a 0)? ¿Podrías explicar por qué?





In [ ]:
# FIX ME